In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import pandas as pd
import os
from os.path import join
import json

In [2]:

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
os.getcwd()

'/cluster/home/kamara/Explain/notebooks'

In [3]:
def parseLogs(logfile, kw="rawlogs:"):
    logs = {}
    dicts = []
    with open(logfile) as f:
        for i_line, line in enumerate(f):
            pos = line.find(kw)
            if pos >= 0:
                pos += len(kw)
                pos_tensor = line.find("tensor(")
                while pos_tensor >= 0:
                    comma_end = line.find(",", pos_tensor)
                    tensor_end = line.find(")", pos_tensor)
                    line_new = line[:pos_tensor] + line[pos_tensor + len("tensor("):comma_end] + line[tensor_end+1:]
                    print("LINE", line)
                    print("NEW", line_new)
                    if len(line_new) >= len(line):
                        print("No line reduction")
                        break
                    line = line_new
                    pos_tensor = line.find("tensor(")
                    
                #print(ast.literal_eval(line[pos:]))
                #line[pos:] = line[pos:].replace("'", "")
                kline = line[pos:]
                kline = kline.replace("true", "True")
                try:
                    dic = ast.literal_eval(kline)
                except:
                    # print("Encountering weird patterns in logs")
                    # print("Line number %d" % i_line)
                    # print(line)
                    line = line.replace("nan,", "-1e8,")
                    line = line.replace("NaN", "-1e8")
                    try:
                        dic = ast.literal_eval(line[pos:])
                    except:
                        print("Unable to replace NaNs")
                        print(line)
                        continue
                for k in dic.keys():
                    if k not in logs:
                        logs[k] = []
                    logs[k].append(dic[k])
                dicts.append(dic)

    df = pd.DataFrame(dicts)

    return logs, df


In [4]:
def get_info(logdir):
    infos, fidelity, initial_mask_infos, transformed_mask_infos = {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, initial_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__initial_mask_infos:')
            _, transformed_mask_infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__transformed_mask_infos:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, initial_mask_infos, transformed_mask_infos, fidelity)


def get_gnn_info(logdir):
    gnn_train, gnn_test = {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, gnn_train[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__gnn_train_scores: ')
            _, gnn_test[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__gnn_test_scores: ')
    return(gnn_train, gnn_test)


def get_param_ranges(jsonpath, dataset = ""):
    f = open(jsonpath,)
    data = json.load(f)
    ranges = data['params']
    ranges['none'] = ''
    return(ranges)   

In [5]:
def get_df_results(logs, ranges, name, metrics, selection = 'last'):
    dicts = []
    for params_set in enumerateParams(ranges):
        key = name.format(**params_set)
        # if key in logs and len(logs[key]) >= 1 and metric in logs[key]:
        any_metric = (key in logs) and (len(logs[key]) >= 1) and any([metric in logs[key] for metric in metrics])
        if any_metric:
            if selection == 'all':
                for index, row in logs[key].iterrows():
                    metrics_dict = {
                        metric: select_value(row.to_frame().T, metric, 'last') if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                        for metric in metrics
                    }
                    dicts.append(dictmerge(params_set, metrics_dict))
                    
            else:
                metrics_dict = {
                    metric: select_value(logs[key], metric, selection) if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                    for metric in metrics
                }
                dicts.append(dictmerge(params_set, metrics_dict))

    df = pd.DataFrame(dicts)
    #df = df.drop(columns=['none'])
    return df

## Node Classification - Real world

### Sparsity

In [25]:
logdir = f'/cluster/home/kamara/Explain/checkpoints/node_classification/real/sparsity_real_avg10expe/logs'
jsonpath = f'/cluster/home/kamara/Explain/configs/config_real_sparsity_avg10expe.json'
infos, initial_mask_infos, transformed_mask_infos, fidelity = get_info(logdir)
gnn_train, gnn_test = get_gnn_info(logdir)

print(list(fidelity.keys())[1])
ranges = get_param_ranges(jsonpath)

_explainer_name=random_sparsity=0_dataset=pubmed_true_label_as_target=True_hard_mask=True_seed=7


In [27]:
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}'
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_true_label_as_target={true_label_as_target}_hard_mask={hard_mask}_seed={seed}'
#name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}'


### GNN scores

In [28]:
df_gnn_train = get_df_results(gnn_train, ranges, name, metrics = ["prec", "recall", "acc"])
df_gnn_test = get_df_results(gnn_test, ranges, name, metrics = ["prec", "recall", "acc"])

df_gnn_train = df_gnn_train.groupby(by=['dataset']).mean().reset_index()
df_gnn_test = df_gnn_test.groupby(by=['dataset']).mean().reset_index()

gnn_scores = pd.merge(df_gnn_train, df_gnn_test, on=['dataset', 'sparsity', 'num_test'], suffixes=['_train', '_test'])
gnn_scores = gnn_scores.drop(columns=["sparsity", "num_test", "seed_train", "seed_test"])
gnn_scores

,dataset,prec_train,recall_train,acc_train,prec_test,recall_test,acc_test
0,actor,0.594775,0.480935,0.538197,0.260749,0.249035,0.285942
1,chameleon,0.817928,0.806324,0.806410,0.646635,0.634489,0.632237
2,citeseer,1.000000,1.000000,1.000000,0.651307,0.652012,0.676400
3,cora,1.000000,1.000000,1.000000,0.781336,0.817381,0.803496
4,cornell,0.927170,0.908066,0.974694,0.354803,0.338859,0.532487
5,facebook,0.931203,0.926196,0.933334,0.923301,0.917619,0.926356
6,pubmed,1.000000,1.000000,1.000000,0.771646,0.781766,0.778500
7,texas,0.972625,0.966848,0.991954,0.296897,0.277057,0.510811
8,wisconsin,0.980174,0.955045,0.975833,0.397882,0.406152,0.535294


In [29]:
#gnn_scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_gnn_scores.csv', index=False)

### Explainability methods scores

In [30]:
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
df_initial_mask_infos = get_df_results(initial_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_tranformed_mask_infos = get_df_results(transformed_mask_infos, ranges, name, metrics = ['mask_size', "mask_entropy", "max_avg"])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 
                                                           'fidelity_gnn_acc+', 'fidelity_gnn_acc-', 'fidelity_gnn_prob+', 'fidelity_gnn_prob-',
                                                           'mask_sparsity', 'expl_edges'])

In [31]:
df_mask_infos = pd.merge(df_initial_mask_infos, df_tranformed_mask_infos, suffixes=['_init', '_transf'], on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores = df_fid
scores = pd.merge(scores, df_mask_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "seed", "sparsity", "true_label_as_target", "hard_mask"])
scores
#scores.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_all.csv', index=False)


,none_x,explainer_name,sparsity,dataset,true_label_as_target,hard_mask,explain_graph_x,num_test,seed,data_save_dir,...,explain_graph_transf,mask_size_transf,mask_entropy_transf,max_avg_transf,none_y,explain_graph_y,time,number_of_edges,mask_sparsity_init,non_zero_values_init
0,,random,0.00,cora,True,True,False,100,0,data,...,False,13249.92,9.299591,0.428312,,False,0.0002,13264,0.001062,13249.92
1,,random,0.00,cora,True,True,False,100,1,data,...,False,13250.48,9.299754,0.490545,,False,0.0002,13264,0.001019,13250.48
2,,random,0.00,cora,True,True,False,100,2,data,...,False,13250.34,9.299545,0.498040,,False,0.0002,13264,0.001030,13250.34
3,,random,0.00,cora,True,True,False,100,3,data,...,False,13249.36,9.299762,0.489739,,False,0.0002,13264,0.001104,13249.36
4,,random,0.00,cora,True,True,False,100,4,data,...,False,13249.54,9.299681,0.470068,,False,0.0002,13264,0.001090,13249.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12327,,pgmexplainer,0.99,facebook,False,True,False,100,1,data,...,False,2768.48,7.475550,0.516261,,False,89.3687,364116,0.944714,20130.39
12328,,pgmexplainer,0.99,facebook,False,True,False,100,2,data,...,False,2392.10,7.094828,0.522513,,False,87.7417,364116,0.948680,18686.40
12329,,pgmexplainer,0.99,facebook,False,True,False,100,3,data,...,False,2672.38,7.429812,0.492495,,False,91.9752,364116,0.957629,15427.86
12330,,pgmexplainer,0.99,facebook,False,True,False,100,4,data,...,False,2674.88,7.356349,0.536693,,False,87.2857,364116,0.951040,17826.97


In [32]:
res = scores.groupby(by=['dataset', "true_label_as_target", "hard_mask", 'sparsity', 'explainer_name']).mean()
res = res.drop(columns=['seed'])
#res[['mask_size', "mask_entropy", "max_avg"]]
res

num_test  \
dataset   true_label_as_target hard_mask sparsity explainer_name                 
actor     False                False     0.000    basic_gnnexplainer     100.0   
                                                  distance               100.0   
                                                  gnnexplainer           100.0   
                                                  gradcam                100.0   
                                                  ig                     100.0   
...                                                                        ...   
wisconsin True                 True      0.997    gnnexplainer           100.0   
                                                  ig                     100.0   
                                                  occlusion              100.0   
                                                  pagerank               100.0   
                                                  random                 100.0   

                                                                      fidelity_acc+  \
dataset   true_label_as_target hard_mask sparsity explainer_name                      
actor     False                False     0.000    basic_gnnexplainer         -1.000   
                                                  distance                   -1.000   
                                                  gnnexplainer               -1.000   
                                                  gradcam                    -1.000   
                                                  ig                         -1.000   
...                                                                             ...   
wisconsin True                 True      0.997    gnnexplainer                0.666   
                                                  ig                          0.366   
                                                  occlusion                   0.667   
                                                  pagerank                    0.290   
                                                  random                      0.228   

                                                                      fidelity_acc-  \
dataset   true_label_as_target hard_mask sparsity explainer_name                      
actor     False                False     0.000    basic_gnnexplainer         -1.000   
                                                  distance                   -1.000   
                                                  gnnexplainer               -1.000   
                                                  gradcam                    -1.000   
                                                  ig                         -1.000   
...                                                                             ...   
wisconsin True                 True      0.997    gnnexplainer                0.458   
                                                  ig                          0.063   
                                                  occlusion                   0.205   
                                                  pagerank                    0.077   
                                                  random                      0.804   

                                                                      fidelity_prob+  \
dataset   true_label_as_target hard_mask sparsity explainer_name                       
actor     False                False     0.000    basic_gnnexplainer       -1.000000   
                                                  distance                 -1.000000   
                                                  gnnexplainer             -1.000000   
                                                  gradcam                  -1.000000   
                                                  ig                       -1.000000   
...                                                                              ...   
wisconsin True                 True

In [33]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/real/nc_real_sparsity_avg10expe.csv', index=False)

## Node Classification - Synthetic

### Sparsity

In [6]:
logdir = f'/cluster/home/kamara/checkpoints/node_classification/sparsity_mask/logs'
jsonpath = f'/cluster/home/kamara/checkpoints/node_classification/sparsity_mask/sweep.json'
infos, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_dataset={dataset}_hard_mask={hard_mask}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
#df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
#scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = df_acc
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores


_explainer_name=random_sparsity=0_dataset=syn3_hard_mask=False


,none_x,explainer_name,sparsity,dataset,hard_mask,explain_graph_x,num_test,data_save_dir,f1_score,recall,...,fidelity_prob+,fidelity_prob-,mask_sparsity,expl_edges,none,explain_graph,time,number_of_edges,mask_sparsity_init,non_zero_values_init
0,,random,0.000,syn1,True,False,100,data,0.005822,1.000000,...,0.511743,-0.000995,0.000243,4109.00,,False,0.0001,4110,0.000243,4109.00
1,,random,0.000,syn1,False,False,100,data,0.005822,1.000000,...,0.229657,0.302100,0.000243,4109.00,,False,0.0001,4110,0.000243,4109.00
2,,random,0.000,syn3,True,False,100,data,0.009404,1.000000,...,0.010216,-0.000004,0.000197,5079.00,,False,0.0001,5080,0.000197,5079.00
3,,random,0.000,syn3,False,False,100,data,0.009404,1.000000,...,-0.001905,-0.001902,0.000197,5079.00,,False,0.0001,5080,0.000197,5079.00
4,,random,0.000,syn4,True,False,100,data,0.012320,1.000000,...,0.963050,0.000000,0.000517,1935.00,,False,0.0000,1936,0.000517,1935.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,,subgraphx,0.997,syn4,False,False,100,data,0.654525,0.516667,...,0.939375,0.959117,0.997417,5.00,,False,0.2794,1936,0.995610,8.50
356,,subgraphx,0.997,syn5,True,False,100,data,0.557429,0.424167,...,0.834673,0.853791,0.997144,9.74,,False,4.7674,3410,0.995595,15.02
357,,subgraphx,0.997,syn5,False,False,100,data,0.560954,0.425833,...,0.837413,0.848950,0.997144,9.74,,False,3.6892,3410,0.995601,15.00
358,,subgraphx,0.997,syn6,True,False,100,data,0.661056,0.634000,...,0.493473,0.183070,0.997829,8.57,,False,0.9027,3948,0.997822,8.60


In [7]:
res = scores.groupby(by=['dataset', 'sparsity', 'explainer_name', 'hard_mask']).mean()
res

num_test  f1_score    recall  \
dataset sparsity explainer_name hard_mask                                 
syn1    0.000    distance       False         100.0  0.005823  1.000000   
                                True          100.0  0.005823  1.000000   
                 gnnexplainer   False         100.0  0.444157  1.000000   
                                True          100.0  0.444157  1.000000   
                 ig             False         100.0  0.005099  0.873333   
...                                             ...       ...       ...   
syn6    0.997    random         True          100.0  0.002500  0.004000   
                 sa             False         100.0  0.000000  0.000000   
                                True          100.0  0.000000  0.000000   
                 subgraphx      False         100.0  0.661056  0.634000   
                                True          100.0  0.661056  0.634000   

                                           precision  fidelity_acc+  \
dataset sparsity explainer_name hard_mask                             
syn1    0.000    distance       False       0.002920           0.28   
                                True        0.002920           0.76   
                 gnnexplainer   False       0.328940           0.56   
                                True        0.328940           0.76   
                 ig             False       0.002557           0.35   
...                                              ...            ...   
syn6    0.997    random         True        0.001818           0.00   
                 sa             False       0.000000           0.00   
                                True        0.000000           0.00   
                 subgraphx      False       0.700500           0.54   
                                True        0.700500           0.54   

                                           fidelity_acc-  fidelity_prob+  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False               0.64        0.102925   
                                True                0.00        0.511743   
                 gnnexplainer   False               0.56        0.296190   
                                True                0.00        0.511743   
                 ig             False               0.64        0.042806   
...                                                  ...             ...   
syn6    0.997    random         True                0.64       -0.000110   
                 sa             False               0.64        0.001296   
                                True                0.64        0.001296   
                 subgraphx      False               0.20        0.493473   
                                True                0.20        0.493473   

                                           fidelity_prob-  mask_sparsity  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False            0.358078       0.010139   
                                True             0.000000       0.010139   
                 gnnexplainer   False            0.348878       0.953431   
                                True             0.000000       0.953431   
                 ig             False            0.412742       0.002487   
...                                                   ...            ...   
syn6    0.997    random         True             0.599028       0.997214   
                 sa             False            0.599028       0.997214   
                                True             0.599028       0.997214   
                 subgraphx      False            0.183070       0.997829   
                                True             0.183070       0.997829   

                                           expl_edges    time  \
dataset sparsity explainer_name hard_mask                       
syn1    0.000    distance       Fal

In [8]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/sparsity_mask_nc.csv', index=False)

### Threshold

In [9]:
logdir = f'/cluster/home/kamara/checkpoints/threshold/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_threshold.json'
infos, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_threshold={threshold}_dataset={dataset}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
#df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
#scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = df_acc
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "threshold"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "threshold"])


_explainer_name=random_threshold=0.5_dataset=syn3


In [10]:
res = scores.groupby(by=['dataset', 'threshold', 'explainer_name']).mean()
res

num_test_nodes  gpu  f1_score    recall  \
dataset threshold explainer_name                                            
syn1    0.5       distance                 100.0  1.0  0.563576  0.413333   
                  gnnexplainer             100.0  1.0  0.365325  0.650000   
                  ig_node                  100.0  1.0  0.446275  0.436667   
                  occlusion                100.0  1.0  0.001630  0.240000   
                  pagerank                 100.0  1.0  0.880128  0.861667   
...                                          ...  ...       ...       ...   
syn6    0.9       pagerank                 100.0  1.0  0.423048  0.314000   
                  pgmexplainer             100.0  1.0  0.380048  0.376000   
                  random                   100.0  1.0  0.005078  0.194000   
                  sa_node                  100.0  1.0  0.180224  0.112000   
                  subgraphx                100.0  1.0  0.773444  0.744000   

                                  precision       auc  fidelity_acc+  \
dataset threshold explainer_name                                       
syn1    0.5       distance         0.939500  0.603303           0.05   
                  gnnexplainer     0.355707  0.720080           0.44   
                  ig_node          0.459524  0.478424           0.05   
                  occlusion        0.000818  0.062967           0.48   
                  pagerank         0.903381  0.930774           0.53   
...                                     ...       ...            ...   
syn6    0.9       pagerank         0.957333  0.656954           0.63   
                  pgmexplainer     0.560708  0.687330           0.57   
                  random           0.002573  0.500879           0.07   
                  sa_node          0.532635  0.336103           0.01   
                  subgraphx        0.811500  0.871812           0.62   

                                  fidelity_acc-  fidelity_prob+  \
dataset threshold explainer_name                                  
syn1    0.5       distance                 0.53       -0.123116   
                  gnnexplainer             0.29        0.285291   
                  ig_node                  0.53        0.024774   
                  occlusion                0.05        0.433345   
                  pagerank                 0.05        0.487905   
...                                         ...             ...   
syn6    0.9       pagerank                 0.36        0.523581   
                  pgmexplainer             0.23        0.488138   
                  random                   0.63        0.060975   
                  sa_node                  0.64       -0.014498   
                  subgraphx                0.05        0.567656   

                                  fidelity_prob-  mask_sparsity  expl_edges  \
dataset threshold explainer_name                                              
syn1    0.5       distance              0.487905       0.999336        2.73   
                  gnnexplainer          0.151176       0.941944      238.61   
                  ig_node               0.445705       0.520219     1971.90   
                  occlusion             0.009594       0.062044     3855.00   
                  pagerank             -0.018173       0.997202       11.50   
...                                          ...            ...         ...   
syn6    0.9       pagerank              0.326768       0.999103        3.54   
                  pgmexplainer          0.253975       0.997219       10.98   
                  random                0.577516       0.899491      396.81   
                  sa_node               0.589573       0.560998     1733.18   
                  subgraphx             0.052988       0.997741        8.92   

                                    time  number_of_edges  mask_sparsity_init  \
dataset threshold explainer_name                                                
syn1    0.5       distance       

In [11]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/threshold.csv', index=False)

### Topk

In [14]:
logdir = f'/cluster/home/kamara/checkpoints/topk/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_topk.json'
infos, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_topk={topk}_dataset={dataset}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
#df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
#scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = df_acc
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "topk"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "topk"])


_explainer_name=random_topk=10_dataset=syn5


In [7]:
res = scores.groupby(by=['dataset', 'topk', 'explainer_name']).mean()
res

num_test_nodes  gpu  f1_score    recall  \
dataset topk explainer_name                                            
syn1    10   distance                 100.0  1.0  0.827099  0.895000   
             gnnexplainer             100.0  1.0  0.393343  0.440000   
             ig_node                  100.0  1.0  0.415758  0.381667   
             occlusion                100.0  1.0  0.040000  0.053333   
             pagerank                 100.0  1.0  0.863636  0.791667   
...                                     ...  ...       ...       ...   
syn6    500  pagerank                 100.0  1.0  0.039216  1.000000   
             pgmexplainer             100.0  1.0  0.056839  0.978000   
             random                   100.0  1.0  0.005317  0.252000   
             sa_node                  100.0  1.0  0.003187  0.112000   
             subgraphx                100.0  1.0  0.765222  0.732000   

                             precision       auc  fidelity_acc+  \
dataset topk explainer_name                                       
syn1    10   distance         0.777849  0.838573           0.38   
             gnnexplainer     0.360433  0.650486           0.21   
             ig_node          0.456667  0.690219           0.05   
             occlusion        0.032000  0.512180           0.25   
             pagerank         0.950000  0.895794           0.53   
...                                ...       ...            ...   
syn6    500  pagerank         0.020000  0.999761           0.64   
             pgmexplainer     0.029418  0.976755           0.64   
             random           0.002687  0.505638           0.06   
             sa_node          0.001619  0.499740           0.01   
             subgraphx        0.813000 -1.000000           0.63   

                             fidelity_acc-  fidelity_prob+  fidelity_prob-  \
dataset topk explainer_name                                                  
syn1    10   distance                 0.33        0.327711        0.226022   
             gnnexplainer             0.52        0.079835        0.415991   
             ig_node                  0.53        0.054472        0.435937   
             occlusion                0.49        0.196120        0.447654   
             pagerank                 0.05        0.487905       -0.005111   
...                                    ...             ...             ...   
syn6    500  pagerank                 0.01        0.589573        0.002842   
             pgmexplainer             0.02        0.589573        0.008439   
             random                   0.60        0.051902        0.568873   
             sa_node                  0.64        0.011466        0.589573   
             subgraphx                0.04        0.573782        0.044031   

                             mask_sparsity  expl_edges    time  \
dataset topk explainer_name                                      
syn1    10   distance             0.997567       10.00  0.0162   
             gnnexplainer         0.997567       10.00  5.4596   
             ig_node              0.997567       10.00  0.1592   
             occlusion            0.997567       10.00  0.5690   
             pagerank             0.997567       10.00  0.0200   
...                                    ...         ...     ...   
syn6    500  pagerank             0.873354      500.00  0.0275   
             pgmexplainer         0.903652      380.38  5.5268   
             random               0.873354      500.00  0.0000   
             sa_node              0.873354      500.00  0.0034   
             subgraphx            0.996484       13.88  3.9086   

                             number_of_edges  mask_sparsity_init  \
dataset topk explainer_name                                        
syn1    10   distance                 4110.0            0.009175   
             gnnexplainer             4110.0            0.937606   
             ig_node                  4110.0            0.050822   
      

In [8]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/topk.csv', index=False)

### Accuracy top

In [15]:
def get_info_top(logdir):
    infos, accuracy_top, accuracy, fidelity = {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, accuracy_top[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy_top:')
            _, accuracy[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, accuracy_top, accuracy, fidelity)


In [16]:
logdir = f'/cluster/home/kamara/checkpoints/top_edges/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_top_edges.json'
infos, accuracy_top, accuracy, fidelity = get_info_top(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_dataset={dataset}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu"])
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu"])


_explainer_name=random_dataset=syn3


In [17]:
res = scores.groupby(by=['dataset', 'explainer_name']).mean()
res

num_test_nodes  gpu  f1_score_top  recall_top  \
dataset explainer_name                                                  
syn1    distance                 200.0  1.0      0.698606    0.686667   
        gnnexplainer             200.0  1.0      0.328182    0.320000   
        ig_node                  200.0  1.0      0.438333    0.438333   
        occlusion                200.0  1.0      0.054167    0.054167   
        pagerank                 200.0  1.0      0.868333    0.868333   
        pgmexplainer             200.0  1.0      0.568561    0.568333   
        random                   200.0  1.0      0.002500    0.002500   
        sa_node                  200.0  1.0      0.018788    0.018333   
        subgraphx                200.0  1.0      0.680818    0.642500   
syn3    distance                 200.0  1.0      0.722813    0.711250   
        gnnexplainer             200.0  1.0      0.610890    0.582083   
        ig_node                  200.0  1.0      0.000000    0.000000   
        occlusion                200.0  1.0      0.332303    0.320833   
        pagerank                 200.0  1.0      0.935674    0.933750   
        pgmexplainer             200.0  1.0      0.644131    0.642917   
        random                   200.0  1.0      0.004583    0.004583   
        sa_node                  200.0  1.0      0.000000    0.000000   
        subgraphx                200.0  1.0      0.568586    0.551667   
syn4    distance                 200.0  1.0      0.626712    0.615000   
        gnnexplainer             200.0  1.0      0.603697    0.588333   
        ig_node                  200.0  1.0      0.858530    0.855000   
        occlusion                200.0  1.0      0.607121    0.605000   
        pagerank                 200.0  1.0      0.839167    0.839167   
        pgmexplainer             200.0  1.0      0.934667    0.913333   
        random                   200.0  1.0      0.006667    0.006667   
        sa_node                  200.0  1.0      0.142727    0.140833   
        subgraphx                200.0  1.0      0.570909    0.549167   
syn5    distance                 200.0  1.0      0.674812    0.658333   
        gnnexplainer             200.0  1.0      0.621695    0.595417   
        ig_node                  200.0  1.0      0.806736    0.797917   
        occlusion                200.0  1.0      0.277803    0.276667   
        pagerank                 200.0  1.0      0.853287    0.851667   
        pgmexplainer             200.0  1.0      0.831922    0.822500   
        random                   200.0  1.0      0.009583    0.009583   
        sa_node                  200.0  1.0      0.835983    0.832083   
        subgraphx                200.0  1.0      0.486891    0.472917   
syn6    distance                 200.0  1.0      0.724667    0.724000   
        gnnexplainer             200.0  1.0      0.204222    0.204000   
        ig_node                  200.0  1.0      0.199222    0.198000   
        occlusion                200.0  1.0      0.023000    0.023000   
        pagerank                 200.0  1.0      0.663556    0.599000   
        pgmexplainer             200.0  1.0      0.381333    0.363000   
        random                   200.0  1.0      0.006000    0.006000   
        sa_node                  200.0  1.0      0.113222    0.109000   
        subgraphx                200.0  1.0      0.679333    0.655000   

                        precision_top  ged  auc_top  f1_score    recall  \
dataset explainer_name                                                    
syn1    distance             0.713333 -1.0     -1.0  0.005823  1.000000   
        gnnexplainer         0.338000 -1.0     -1.0  0.447901  1.000000   
        ig_node              0.438333 -1.0     -1.0  0.007163  0.915833   
        occlusion            0.054167 -1.0     -1.0  0.005822  1.000000   
        pagerank             0.868333 -1.0     -1.0  0.005825  1.000000   
        pgmexplainer         0.568833 -1.0     -1.0  0.164252  1.00

In [18]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/top_edges.csv', index=False)

### Gc Layers

In [50]:
expe = "gc_layers"
logdir = f'/cluster/home/kamara/checkpoints/{expe}/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy, fidelity = get_info(logdir)
ranges = get_param_ranges(jsonpath)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_num_gc_layers={num_gc_layers}'


_explainer_name=pagerank_num_gc_layers=1


In [51]:
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-'])
scores = pd.merge(df_acc, df_fid, on=["dataset", "num_gc_layers", "explainer_name", "num_test_nodes", "data_save_dir"])
scores = scores.groupby(by=['num_gc_layers', 'explainer_name']).mean()
scores

num_test_nodes  f1_score    recall  precision  \
num_gc_layers explainer_name                                                  
2             gnnexplainer             200.0  0.682879  0.652500   0.720833   
              pagerank                 200.0  0.864167  0.864167   0.864167   
3             gnnexplainer             200.0  0.329136  0.323333   0.336250   
              pagerank                 200.0  0.864167  0.864167   0.864167   
4             gnnexplainer             200.0  0.055833  0.055833   0.055833   
              pagerank                 200.0  0.860000  0.860000   0.860000   
5             gnnexplainer             200.0  0.006667  0.006667   0.006667   
              pagerank                 200.0  0.859167  0.859167   0.859167   
6             gnnexplainer             200.0  0.003333  0.003333   0.003333   
              pagerank                 200.0  0.859167  0.859167   0.859167   

                                 ged       auc  fidelity_acc+  fidelity_acc-  \
num_gc_layers explainer_name                                                   
2             gnnexplainer     4.065  0.998892          0.835          0.000   
              pagerank         2.435  0.999885          0.835          0.000   
3             gnnexplainer     8.160  0.972214          0.610          0.015   
              pagerank         2.430  0.999885          0.610          0.000   
4             gnnexplainer    12.475  0.834369          0.750          0.290   
              pagerank         2.515  0.999885          0.830          0.000   
5             gnnexplainer    14.085  0.656783          0.550          0.575   
              pagerank         2.525  0.999883          0.640          0.005   
6             gnnexplainer    14.345  0.575590          0.610          0.645   
              pagerank         2.525  0.999883          0.640          0.005   

                              fidelity_prob+  fidelity_prob-  
num_gc_layers explainer_name                                  
2             gnnexplainer          0.650817        0.000000  
              pagerank              0.650817        0.000000  
3             gnnexplainer          0.582833        0.014268  
              pagerank              0.589879       -0.000021  
4             gnnexplainer          0.571764        0.246716  
              pagerank              0.640806        0.005695  
5             gnnexplainer          0.512901        0.548674  
              pagerank              0.608210       -0.001958  
6             gnnexplainer          0.545983        0.581469  
              pagerank              0.579183       -0.000794

## Graph Classification

### Sparsity

In [13]:
logdir = f'/cluster/home/kamara/checkpoints/graph_classification/sparsity_mask/logs'
jsonpath = f'/cluster/home/kamara/checkpoints/graph_classification/sparsity_mask/sweep.json'
infos, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}_hard_mask={hard_mask}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', "number_of_edges", "mask_sparsity_init", "non_zero_values_init"])
#df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
#scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = df_acc
print(scores)
scores = pd.merge(scores, df_fid, on=["explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores = pd.merge(scores, df_infos, on=["explainer_name", "num_test", "data_save_dir", "sparsity", "hard_mask"])
scores


_explainer_name=random_sparsity=0.7_hard_mask=True
Empty DataFrame
Columns: []
Index: []


KeyError: 'explainer_name'

In [7]:
res = scores.groupby(by=['dataset', 'sparsity', 'explainer_name', 'hard_mask']).mean()
res

num_test  f1_score    recall  \
dataset sparsity explainer_name hard_mask                                 
syn1    0.000    distance       False         100.0  0.005823  1.000000   
                                True          100.0  0.005823  1.000000   
                 gnnexplainer   False         100.0  0.444157  1.000000   
                                True          100.0  0.444157  1.000000   
                 ig             False         100.0  0.005099  0.873333   
...                                             ...       ...       ...   
syn6    0.997    random         True          100.0  0.002500  0.004000   
                 sa             False         100.0  0.000000  0.000000   
                                True          100.0  0.000000  0.000000   
                 subgraphx      False         100.0  0.661056  0.634000   
                                True          100.0  0.661056  0.634000   

                                           precision  fidelity_acc+  \
dataset sparsity explainer_name hard_mask                             
syn1    0.000    distance       False       0.002920           0.28   
                                True        0.002920           0.76   
                 gnnexplainer   False       0.328940           0.56   
                                True        0.328940           0.76   
                 ig             False       0.002557           0.35   
...                                              ...            ...   
syn6    0.997    random         True        0.001818           0.00   
                 sa             False       0.000000           0.00   
                                True        0.000000           0.00   
                 subgraphx      False       0.700500           0.54   
                                True        0.700500           0.54   

                                           fidelity_acc-  fidelity_prob+  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False               0.64        0.102925   
                                True                0.00        0.511743   
                 gnnexplainer   False               0.56        0.296190   
                                True                0.00        0.511743   
                 ig             False               0.64        0.042806   
...                                                  ...             ...   
syn6    0.997    random         True                0.64       -0.000110   
                 sa             False               0.64        0.001296   
                                True                0.64        0.001296   
                 subgraphx      False               0.20        0.493473   
                                True                0.20        0.493473   

                                           fidelity_prob-  mask_sparsity  \
dataset sparsity explainer_name hard_mask                                  
syn1    0.000    distance       False            0.358078       0.010139   
                                True             0.000000       0.010139   
                 gnnexplainer   False            0.348878       0.953431   
                                True             0.000000       0.953431   
                 ig             False            0.412742       0.002487   
...                                                   ...            ...   
syn6    0.997    random         True             0.599028       0.997214   
                 sa             False            0.599028       0.997214   
                                True             0.599028       0.997214   
                 subgraphx      False            0.183070       0.997829   
                                True             0.183070       0.997829   

                                           expl_edges    time  \
dataset sparsity explainer_name hard_mask                       
syn1    0.000    distance       Fal

In [8]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/node_classification/sparsity_mask_nc.csv', index=False)